<a href="https://colab.research.google.com/github/regain001/Data-Extraction-From-PDF/blob/master/data-extraction-from-pdf(python-lib).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extract pdf data by using python**



In [0]:
!pip install PyPDF2
!pip install tika
!pip install py4j
!pip install tabula-py

In [0]:
basePath = '/content/drive/My Drive/NazTech/bank pdf files/'

### Using PyPDF2

In [0]:
import PyPDF2
from google.colab import drive

pdfFileObj = open(basePath+'ABSABank.pdf', 'rb')

pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print(pdfReader.numPages)

pageObj = pdfReader.getPage(0)
print(pageObj.extractText())


## Comment : need to regex after getting the texts

9
Transaction History (2020-04-28 16:05:03)SOUTHEAST EXCHANGE COMPANY SOUTH AFRIC (PTY) LTDABSA9 DOLLY RATHEBE ROAD4095449786FORDSBURGClient Account2092  Current balanceR 852.90Available Balance as atR 852.90Uncleared ChequesR 0.00 Statement for Period 2020-04-20 - 2020-04-28 Date:Transaction DescriptionAmountBalanceBalance Brought Forward51,372.902020-04-20CARDLESS CASH DEP   CLEARWATER(     21,55 ) DEPOSIT NO : 102367 CONTACT : 08485496275,130.0056,502.902020-04-20CARDLESS CASH DEP   CLEARWATER(     21,55 ) DEPOSIT NO : 102369 CONTACT : 08485496275,130.0061,632.902020-04-20CARDLESS CASH DEP   CLEARWATER(     21,55 ) DEPOSIT NO : 1028283 CONTACT : 08485496275,130.0066,762.902020-04-20CARDLESS CASH DEP   CLEARWATER(     21,55 ) DEPOSIT NO : 1028021 CONTACT : 08485496275,130.0071,892.902020-04-20CARDLESS CASH DEP   EDENBURG 1(     21,55 ) DEPOSIT NO : 1028596 CONTACT : 07470441635,130.0077,022.902020-04-20CARDLESS CASH DEP   EDENBURG 1(     21,55 ) DEPOSIT NO : 1021293 CONTACT : 0747044

### Using Tica

In [0]:
from tika import parser

raw = parser.from_file(basePath+'ABSABank.pdf')
raw = str(raw)

safe_text = raw.encode('utf-8', errors='ignore')

safe_text = str(safe_text).replace("\n", "").replace("\\", "")
print('--- safe text ---' )
print( safe_text )


## Comment : need to regex after getting the texts 
## & need to parse from this variable -- raw['content'] 

--- safe text ---
b'{'metadata': {'Content-Type': 'application/pdf', 'Creation-Date': '2020-04-28T14:05:03Z', 'Last-Modified': '2020-04-28T14:05:03Z', 'Last-Save-Date': '2020-04-28T14:05:03Z', 'X-Parsed-By': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'X-TIKA:embedded_depth': '0', 'X-TIKA:parse_time_millis': '258', 'access_permission:assemble_document': 'true', 'access_permission:can_modify': 'true', 'access_permission:can_print': 'true', 'access_permission:can_print_degraded': 'true', 'access_permission:extract_content': 'true', 'access_permission:extract_for_accessibility': 'true', 'access_permission:fill_in_form': 'true', 'access_permission:modify_annotations': 'true', 'created': '2020-04-28T14:05:03Z', 'date': '2020-04-28T14:05:03Z', 'dc:format': 'application/pdf; version=1.5', 'dcterms:created': '2020-04-28T14:05:03Z', 'dcterms:modified': '2020-04-28T14:05:03Z', 'meta:creation-date': '2020-04-2

### Using py4j

In [0]:
from py4j.java_gateway import JavaGateway

gw = JavaGateway()
result = gw.entry_point.strip(r'E:\NazTech\OFFICE WORK\statements\ABSABank.pdf')

# result is a dict of {
#   'success': 'true' or 'false',
#   'payload': pdf file content if 'success' is 'true'
#   'error': error message if 'success' is 'false'
# }

print (result['payload'])


## Comment : couldn't run  
## & need to download a jar to see the results

### Using tabula with pandas

In [0]:
import tabula
import pandas as pd
import re
# df = tabula.read_pdf(basePath+'ABSABank.pdf', area=(210,0,250,400), pages=1)
# df = tabula.read_pdf(path, pages = '1', )
df = tabula.read_pdf(basePath+'ABSABank.pdf', area=(0,0,350,500), pages=1)

print(df)

Got stderr: May 06, 2020 7:30:29 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
May 06, 2020 7:30:29 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



[       Transaction History (2020-04-28 16:05:03)      Unnamed: 0 Unnamed: 1
0   SOUTHEAST EXCHANGE COMPANY SOUTH AFRIC (PTY)            ABSA        NaN
1                                            LTD             NaN        NaN
2                           9 DOLLY RATHEBE ROAD      4095449786        NaN
3                                      FORDSBURG  Client Account        NaN
4                                           2092             NaN        NaN
5                                            NaN             NaN        NaN
6                                Current balance        R 852.90        NaN
7                        Available Balance as at        R 852.90        NaN
8                              Uncleared Cheques          R 0.00        NaN
9                                            NaN             NaN        NaN
10  Statement for Period 2020-04-20 - 2020-04-28             NaN        NaN
11                                           NaN             NaN        NaN
12         

In [0]:
# print("Bank Name   :",df[0].values[0][1].replace(' ',''))

curBal = df[0].values[6][1].replace('R ','')
print("Current balance   :", curBal)

avaBal = df[0].values[7][1].replace('R ','')
print("Available balance :",avaBal)

## But need to use regex after getting this string.
date_range = df[0].values[10][0]
date = re.search("(?<=Statement for Period \d{4}-\d{2}-\d{2}( )-( ))\d{4}-\d{2}-\d{2}", date_range)

print("Date : "+str(date.group()))

Current balance   : 852.90
Available balance : 852.90
Date : 2020-04-28


In [0]:
# actual output after converting to json
df = tabula.read_pdf(basePath+'ABSABank.pdf', area=(0,0,350,500), pages=1, output_format="json")
print(df)

Got stderr: May 06, 2020 7:26:27 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
May 06, 2020 7:26:28 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



[{'extraction_method': 'stream', 'top': 0.0, 'left': 0.0, 'width': 500.0, 'height': 350.0, 'right': 500.0, 'bottom': 350.0, 'data': [[{'top': 95.72, 'left': 19.86, 'width': 236.75, 'height': 5.550000190734863, 'text': 'Transaction History (2020-04-28 16:05:03)'}, {'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''}, {'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''}], [{'top': 132.16, 'left': 19.8, 'width': 277.7900085449219, 'height': 4.630000114440918, 'text': 'SOUTHEAST EXCHANGE COMPANY SOUTH AFRIC (PTY)'}, {'top': 132.16, 'left': 305.3, 'width': 28.330015182495117, 'height': 4.630000114440918, 'text': 'ABSA'}, {'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''}], [{'top': 143.91, 'left': 19.8, 'width': 19.440000534057617, 'height': 4.630000114440918, 'text': 'LTD'}, {'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''}, {'top': 0.0, 'left': 0.0, 'width': 0.0, 'height': 0.0, 'text': ''}], [{'top': 158.66, 'left': 19.8, 'width

In [0]:
# Or we can make a json

import json

dt = str(date.group())
jsnData = {
  'currentBalance': curBal,
  'availableBalance': avaBal,
  'date': dt
}
jsn = json.dumps(jsnData)
print(jsn)

{"currentBalance": "852.90", "availableBalance": "852.90", "date": "2020-04-28"}


In [0]:
## If the informations locate inside the header, then we can get it easily 
## without using regex. 
## There are many ways of getting data from pdf using python, but I dont know 
## if there is any better solution to extract data by open source library